# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json 
# import gmaps.datasets

# Import API key
from config import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# CSV from WeatherPy
csv_file = pd.read_csv("../WeatherPy/output_data/cities.csv") 

vacay_df = pd.DataFrame(csv_file)

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
#  configure gmaps
gmaps.configure(api_key=g_key)

In [4]:
# vacay_df.head()

# Use the Lat and Lng as locations 
locations = vacay_df[['Lat', 'Lng']]

# Use Humidity as the weight.
humidity = vacay_df['Humidity (%)'].astype(float)


In [5]:
# Customize the size of the figure
figure_layout = {
    'width': '600px',
    'height': '500px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}

# Plot Heatmap
fig = gmaps.figure(g_key)

# # Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity)


# # Add layer
fig.add_layer(heat_layer)

# # Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [6]:
# Max temperature lower than 80 degrees but higher than 70.
# Wind speed less than 10 mph.
# Zero cloudiness.
# Drop any rows that don't contain all three conditions. You want to be sure the weather is ideal.
# vacay_df

# use .loc to create df with ideal weather conditions
ideal_df = vacay_df.loc[(vacay_df["Max Temp (F)"] < 80) & (vacay_df["Max Temp (F)"] > 70) & 
                        (vacay_df["Wind Speed (mph)"] < 10) & (vacay_df["Cloudiness (%)"] == 0)]
ideal_df

,Unnamed: 0,City,Lat,Lng,Max Temp (F),Humidity (%),Cloudiness (%),Wind Speed (mph),Country,Date
3,3,tarija,-21.54,-64.73,75.20,31,0,5.82,BO,1596650871
170,170,manga,-33.70,-53.46,71.46,67,0,9.24,UY,1596650930
199,199,tateyama,-24.79,-50.01,72.68,36,0,3.47,BR,1596650791
239,239,mtwara,40.56,8.32,78.80,57,0,5.82,IT,1596650853
249,249,xining,-26.62,-54.13,77.40,35,0,5.93,AR,1596650956
375,375,carutapera,45.25,77.98,79.75,27,0,9.84,KZ,1596651004
385,385,druzhba,-16.23,39.91,72.18,76,0,7.63,MZ,1596651007
388,388,nesbyen,-19.88,-44.98,77.67,29,0,8.32,BR,1596650995
407,407,ancud,-9.35,28.73,70.88,53,0,4.00,ZM,1596651016
431,431,burica,45.44,40.58,73.44,54,0,7.02,RU,1596651026


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [7]:
# Store into variable named hotel_df
hotel_df = ideal_df

# add empty "Hotel Name" column
hotel_df["Hotel Name"] = ""
# hotel_df.count()
hotel_df


/Applications/anaconda3/envs/PythonData/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


,Unnamed: 0,City,Lat,Lng,Max Temp (F),Humidity (%),Cloudiness (%),Wind Speed (mph),Country,Date,Hotel Name
3,3,tarija,-21.54,-64.73,75.20,31,0,5.82,BO,1596650871,
170,170,manga,-33.70,-53.46,71.46,67,0,9.24,UY,1596650930,
199,199,tateyama,-24.79,-50.01,72.68,36,0,3.47,BR,1596650791,
239,239,mtwara,40.56,8.32,78.80,57,0,5.82,IT,1596650853,
249,249,xining,-26.62,-54.13,77.40,35,0,5.93,AR,1596650956,
375,375,carutapera,45.25,77.98,79.75,27,0,9.84,KZ,1596651004,
385,385,druzhba,-16.23,39.91,72.18,76,0,7.63,MZ,1596651007,
388,388,nesbyen,-19.88,-44.98,77.67,29,0,8.32,BR,1596650995,
407,407,ancud,-9.35,28.73,70.88,53,0,4.00,ZM,1596651016,
431,431,burica,45.44,40.58,73.44,54,0,7.02,RU,1596651026,


In [9]:
# target_coordinates = 
target_search = "Hotel"
target_radius = 5000
target_type = "lodging"

for index, row in hotel_df.iterrows():
    params = {
    "location": f"{row['Lat']}, {row['Lng']}",
    "keyword": target_search,
    "radius": target_radius,
    "type": target_type,
    "key": g_key
    } 
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    response = requests.get(base_url, params=params).json()
    
#     extract results
    results = response['results']
    
    try:
        print(f"The Hotel to stay at is {results[0]['name']}")
        
        hotel_df.loc[index, "Hotel Name"] = results[0]['name']
    except (KeyError, IndexError):
        print("Missing field/result...skipping")

The Hotel to stay at is Hotel Los Parrales


/Applications/anaconda3/envs/PythonData/lib/python3.6/site-packages/pandas/core/indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


The Hotel to stay at is Bertelli Chuí Hotel
The Hotel to stay at is Urano Business Hotel
The Hotel to stay at is Villa Las Tronas
The Hotel to stay at is Hotel Y Restaurante Los Lapachos
Missing field/result...skipping
The Hotel to stay at is Guest House HESADA
The Hotel to stay at is Liber Hotel e Restaurante
The Hotel to stay at is CM Guest house
The Hotel to stay at is Luxury House ECO-hotel & Spa
The Hotel to stay at is Ilha Flat Hotel
Missing field/result...skipping


In [11]:
# drop null values
# hotel_df
clean_hotel_df = hotel_df.loc[hotel_df["Hotel Name"] != ""]
clean_hotel_df

,Unnamed: 0,City,Lat,Lng,Max Temp (F),Humidity (%),Cloudiness (%),Wind Speed (mph),Country,Date,Hotel Name
3,3,tarija,-21.54,-64.73,75.20,31,0,5.82,BO,1596650871,Hotel Los Parrales
170,170,manga,-33.70,-53.46,71.46,67,0,9.24,UY,1596650930,Bertelli Chuí Hotel
199,199,tateyama,-24.79,-50.01,72.68,36,0,3.47,BR,1596650791,Urano Business Hotel
239,239,mtwara,40.56,8.32,78.80,57,0,5.82,IT,1596650853,Villa Las Tronas
249,249,xining,-26.62,-54.13,77.40,35,0,5.93,AR,1596650956,Hotel Y Restaurante Los Lapachos
385,385,druzhba,-16.23,39.91,72.18,76,0,7.63,MZ,1596651007,Guest House HESADA
388,388,nesbyen,-19.88,-44.98,77.67,29,0,8.32,BR,1596650995,Liber Hotel e Restaurante
407,407,ancud,-9.35,28.73,70.88,53,0,4.00,ZM,1596651016,CM Guest house
431,431,burica,45.44,40.58,73.44,54,0,7.02,RU,1596651026,Luxury House ECO-hotel & Spa
515,515,todos santos,-23.78,-45.36,70.45,66,0,7.11,BR,1596651055,Ilha Flat Hotel


In [12]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [13]:
# # Add marker layer ontop of heat map
hotel_markers = gmaps.marker_layer(locations, info_box_content=hotel_info)

figure_layout = {
    'width': '600px',
    'height': '500px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
fig = gmaps.figure(layout=figure_layout)
fig.add_layer(heat_layer)
fig.add_layer(hotel_markers)

# # Display figure
fig


Figure(layout=FigureLayout(border='1px solid black', height='500px', margin='0 auto 0 auto', padding='1px', wi…